<div class="alert alert-block alert-success">
    <h1>
        Example notebook - Reactome subgraph
    </h1>
</div>

# Import modules and functions

In [1]:
import os
import glob
import re
import networkx as nx
import pandas as pd
import time
from tqdm.auto import tqdm

from IPython.display import display, Markdown

from turingdb_examples.graph import networkx_to_jsonl
from turingdb_examples.llm import natural_language_to_cypher, query_llm

In [2]:
%load_ext autoreload
%autoreload 2

# Check data files are available

In [3]:
example_name = "reactome"
path_data = f"{os.getcwd()}/data/{example_name}"
if not os.path.exists(path_data):
    raise ValueError(f"{path_data} does not exists")

list_csv_files = sorted(
    [os.path.basename(file) for file in glob.glob(os.path.join(path_data, "*"))]
)
if not "entities_pairwise.gml" in list_csv_files:
    raise ValueError(
        f"At least one of the {len(list_csv_files)} csv files is not available in {path_data}"
    )

# Import `gml` file

In [4]:
G = nx.read_gml(f"{path_data}/entities_pairwise.gml")
print(G)

MultiGraph with 40 nodes and 57 edges


# Create `turingdb` python client

<div class="alert alert-block alert-info">
    <h2>
        See <a href="https://docs.turingdb.ai/quickstart">TuringDB Get started documentation</a> for the important steps to follow :
    </h2>
    <h3>
        Remember to have your <code>turingdb</code> server running while working in this notebook !
    </h3>
</div>

In [5]:
from turingdb import TuringDB

# Create TuringDB client
# set host parameter to the URL (as string) on which TuringDB is running,
# default "http://localhost:6666"
client = TuringDB(host="http://localhost:6666")
try:
    client.warmup()
except Exception as e:
    print(f"TuringDB not started, please run `uv run turingdb` in your terminal")

In [6]:
# Get list of available graphs
list_graphs = client.list_available_graphs()

In [7]:
# Get list of loaded graphs
client.list_loaded_graphs()

['london_transport_TfL1_subgraph',
 'london_transport_TfL1',
 'wine_ontology1',
 'healthcare_dataset1_subgraph',
 'healthcare_dataset1',
 'paysim_financial_fraud_detection1',
 'crypto_orbitaal_fraud_detection1',
 'default']

# Set graph name

In [8]:
# Set graph name
graph_name_prefix = example_name
graph_name_nb_suffix = str(
    max(
        [
            int(re.sub(graph_name_prefix, "", g))
            for g in list_graphs
            if g.startswith(graph_name_prefix)
            and re.sub(graph_name_prefix, "", g).isdigit()
        ]
        + [0]
    )
    + 1
)
graph_name = graph_name_prefix + graph_name_nb_suffix
graph_name = re.sub("-", "_", graph_name)
graph_name

'reactome3'

# Create JSONL file

In [10]:
turingdb_dir = os.path.expanduser("~/.turing/data")
if not os.path.exists(turingdb_dir):
    raise ValueError(f"""
    TuringDB directory {turingdb_dir} does not exist.
    Make sure the value you set here is the same you set when running turingdb.
    """)

In [11]:
%%time

# Write graph to JSONL format
print("Writing graph to JSONL...")
jsonl_filename = networkx_to_jsonl(
    G,
    graph_name,
    node_type_key="schemaClass",
    data_dir=turingdb_dir
)

Writing graph to JSONL...
JSONL file written to: /home/ubuntu/.turing/data/reactome3.jsonl
Graph: 40 nodes, 57 edges
CPU times: user 997 μs, sys: 0 ns, total: 997 μs
Wall time: 842 μs


# Create `turingdb` graph

In [12]:
print(f"Creating graph: {graph_name}")

Creating graph: reactome3


In [13]:
%%time

start_time = time.time()

# Load JSONL into TuringDB
print(f"\nLoading JSONL into TuringDB as graph '{graph_name}' ...")
client.query(f"LOAD JSONL '{jsonl_filename}' AS {graph_name}")

execution_time = time.time() - start_time
print(f"\n\u2713 Graph '{graph_name}' created in {execution_time:.2f} seconds")

# Set active graph for querying
client.set_graph(graph_name)


Loading JSONL into TuringDB as graph 'reactome3' ...

✓ Graph 'reactome3' created in 0.07 seconds
CPU times: user 164 μs, sys: 1.99 ms, total: 2.15 ms
Wall time: 67.6 ms


In [14]:
# Returns the commit history
client.query("CALL db.history()")

,commit,nodeCount,edgeCount,partCount
0,f3cb9c062ae9c661,0,0,0
1,425c60854574202,40,57,1


# Query `turingdb` graph

## Use metaqueries to have insight on graph overall structure

<h3>
    To learn more about 📮 Metaqueries, please check TuringDB documentation on this <a href="https://turingdb.mintlify.app/query/cypher_subset#%F0%9F%93%AE-metaqueries">link</a>
</h3>

In [17]:
%%time

# CALL propertyTypes() - returns a column of all the different node and edge properties and their types in the database
command = """
CALL db.propertyTypes()
"""
df_propertyTypes = client.query(command)
if df_propertyTypes.empty:
    print("No result found")
else:
    display(df_propertyTypes)

,id,propertyType,valueType
0,0,category,String
1,1,displayName,String
2,2,id,String
3,3,name,String
4,4,oldStId,String
5,5,releaseDate,String
6,6,schemaClass,String
7,7,speciesName,String
8,8,stId,String
9,9,stIdVersion,String


CPU times: user 3.07 ms, sys: 2 μs, total: 3.07 ms
Wall time: 2.95 ms


In [18]:
# Get node properties
nodes_properties = df_propertyTypes["propertyType"].values.tolist()
print(f"Node properties: {nodes_properties}")

Node properties: ['category', 'displayName', 'id', 'name', 'oldStId', 'releaseDate', 'schemaClass', 'speciesName', 'stId', 'stIdVersion', 'referenceType', 'searchSeed']


In [19]:
%%time

# CALL labels () - returns a column of all the different node labels
command = """
CALL db.labels()
"""
df_labels = client.query(command)
if df_labels.empty:
    print("No result found")
else:
    display(df_labels)

,id,label
0,0,Reaction
1,1,Complex
2,2,BlackBoxEvent
3,3,EntityWithAccessionedSequence
4,4,PositiveGeneExpressionRegulation
5,5,PositiveRegulation


CPU times: user 1.94 ms, sys: 949 μs, total: 2.89 ms
Wall time: 2.81 ms


In [20]:
%%time

# CALL edgeTypes() - returns a column of all the different edge types (edge equivalent of node labels)
command = """
CALL db.edgeTypes()
"""
df_edgeTypes = client.query(command)
if df_edgeTypes.empty:
    print("No result found")
else:
    display(df_edgeTypes)

,id,edgeType
0,0,CONNECTED


CPU times: user 2.65 ms, sys: 37 μs, total: 2.69 ms
Wall time: 2.5 ms


## Counts

In [21]:
%%time

# Find number of nodes and number of edges in the graph
n_nodes = len(client.query("MATCH (n) RETURN n"))
n_edges = len(client.query("MATCH (n)-->(m) RETURN n, m"))
print(f"Graph: {n_nodes:,} nodes and {n_edges:,} edges\n")

Graph: 40 nodes and 57 edges

CPU times: user 2.12 ms, sys: 0 ns, total: 2.12 ms
Wall time: 1.94 ms


In [22]:
%%time

# Count all nodes
command = """
MATCH (n)
RETURN COUNT(n)
"""
df_count_nodes = client.query(command)
display(df_count_nodes)

# Count all edges
command = """
MATCH (n)-->()
RETURN COUNT(n)
"""
df_count_edges = client.query(command)
display(df_count_edges)

# Find number of nodes and number of edges in the graph
n_nodes = int(df_count_nodes.loc[0, "COUNT(n)"])
n_edges = int(df_count_edges.loc[0, "COUNT(n)"])
print(f"Graph: {n_nodes:,} nodes and {n_edges:,} edges\n")

,COUNT(n)
0,40


,COUNT(n)
0,57


Graph: 40 nodes and 57 edges

CPU times: user 4.55 ms, sys: 0 ns, total: 4.55 ms
Wall time: 4.33 ms


In [23]:
# Count number of nodes for each label
for label in df_labels["label"]:
    print(100 * '-')
    print(f"label: {label}")
    df_curr_label = client.query(f"""
    MATCH (n:{label})
    RETURN n.name
    """)
    df_curr_label_count = client.query(f"""
    MATCH (n:{label})
    RETURN count(n)
    """)
    display(df_curr_label)
    display(df_curr_label_count)
    
    print()
print(100 * '-')

----------------------------------------------------------------------------------------------------
label: Reaction


,n.name
0,['TP53 binds the PMAIP1 (NOXA) promoter']
1,['TP53 binds the APAF1 gene promoter']
2,['NRF1:PPARGC1B binds the CYCS promoter']
3,['CYCS binds to APAF1']
4,['E2F1 binds APAF1 gene promoter']
5,['p38 MAPK phosphorylates PPARGC1A']
6,"['NRF1:p-PPARGC1A, NRF2 bind the TFB2M promoter']"
7,['E2F1 binds PMAIP1 (NOXA) promoter']
8,['Translocation of PMAIP1 (NOXA) to mitochondr...
9,['BH3-only proteins associate with and inactiv...


,count(n)
0,12



----------------------------------------------------------------------------------------------------
label: Complex


,n.name
0,"['E2F1:(TFDP1,TFDP2)']"
1,"['APAF1:CYCS','APAF1:Cytochrome C']"
2,"['CYCS gene:NRF1:PPARGC1B','NRF1:PGC-1beta:CYCS']"
3,['RORA:Coactivator']
4,"['p-S15,S20-TP53:EP300:PRMT1:CARM1:GADD45A Gene']"
5,['ESRRA:PPARGC1A']
6,"['p-S15,S20-TP53 Tetramer']"
7,"['p-S15,S20-TP53 Tetramer:PMAIP1 Gene']"
8,"['E2F1:TFDP1,TFDP2','DP1/2:E2F1']"


...

,count(n)
0,2



----------------------------------------------------------------------------------------------------
label: PositiveRegulation


,n.name
0,<NA>


,count(n)
0,1



----------------------------------------------------------------------------------------------------


## Queries

In [24]:
%%time

# Match all edges and return them
command = """
MATCH (n)-[e]->(m)
RETURN n.displayName, e, m.displayName
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,n.displayName,e,m.displayName
0,TP53 binds the PMAIP1 (NOXA) promoter,0,"p-S15,S20-TP53 Tetramer [nucleoplasm]"
1,TP53 binds the PMAIP1 (NOXA) promoter,1,PMAIP1 Gene [nucleoplasm]
2,TP53 binds the PMAIP1 (NOXA) promoter,2,"p-S15,S20-TP53 Tetramer:PMAIP1 Gene [nucleoplasm]"
3,TP53 binds the PMAIP1 (NOXA) promoter,3,TP53 stimulates PMAIP1 (NOXA) expression
4,TP53 binds the APAF1 gene promoter,4,TP53 stimulates APAF1 gene expression
5,NRF1:PPARGC1B binds the CYCS promoter,5,Expression of CYCS
6,NRF1:PPARGC1B binds the CYCS promoter,6,Expression of NRF1
7,CYCS binds to APAF1,7,Release of Cytochrome c from mitochondria
8,CYCS binds to APAF1,8,CYCS [cytosol]
9,E2F1 binds APAF1 gene promoter,9,APAF1 gene expression is stimulated by E2F1 an...


CPU times: user 3.81 ms, sys: 46 μs, total: 3.85 ms
Wall time: 3.72 ms


In [25]:
%%time

# Find all nodes of type "EntityWithAccessionedSequence" and referenceType "ReferenceGeneProduct"
command = """
MATCH (n:EntityWithAccessionedSequence)
WHERE n.referenceType = "ReferenceGeneProduct"
RETURN n.displayName, n.schemaClass, n.referenceType
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,n.displayName,n.schemaClass,n.referenceType
0,E2F1 [nucleoplasm],EntityWithAccessionedSequence,ReferenceGeneProduct
1,ESRRA [nucleoplasm],EntityWithAccessionedSequence,ReferenceGeneProduct
2,CYCS [cytosol],EntityWithAccessionedSequence,ReferenceGeneProduct
3,EP300 [nucleoplasm],EntityWithAccessionedSequence,ReferenceGeneProduct
4,PPARGC1A [nucleoplasm],EntityWithAccessionedSequence,ReferenceGeneProduct
5,NRF1 [nucleoplasm],EntityWithAccessionedSequence,ReferenceGeneProduct
6,CYCS [mitochondrial intermembrane space],EntityWithAccessionedSequence,ReferenceGeneProduct
7,PMAIP1 [cytosol],EntityWithAccessionedSequence,ReferenceGeneProduct
8,APAF1 [cytosol],EntityWithAccessionedSequence,ReferenceGeneProduct


CPU times: user 2.99 ms, sys: 0 ns, total: 2.99 ms
Wall time: 2.84 ms


In [26]:
%%time

# Count nodes of each schemaClass
command = """
MATCH (n)
RETURN n.schemaClass
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(pd.DataFrame(df.value_counts()))

,count
n.schemaClass,
Reaction,12
EntityWithAccessionedSequence,10
Complex,9
BlackBoxEvent,6
PositiveGeneExpressionRegulation,2
PositiveRegulation,1


CPU times: user 3.42 ms, sys: 24 μs, total: 3.44 ms
Wall time: 3.28 ms


In [27]:
%%time

# Find all nodes of type "PositiveGeneExpressionRegulation" (which is also schemaClass)
command = """
MATCH (n:PositiveGeneExpressionRegulation)
RETURN n, n.displayName, n.schemaClass
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,n,n.displayName,n.schemaClass
0,37,"Positive gene expression regulation by 'p-S15,...",PositiveGeneExpressionRegulation
1,38,Positive gene expression regulation by 'E2F1:T...,PositiveGeneExpressionRegulation


CPU times: user 2.81 ms, sys: 48 μs, total: 2.86 ms
Wall time: 2.64 ms


In [28]:
%%time

# Find all edges involving a node of type Complex (undirected edge get both directions of directed edges)
command = """
MATCH (n:Complex)-[e]-(m)
RETURN n.displayName, n.schemaClass, e, m.displayName, m.schemaClass, m.category
"""
df = client.query(command)

df

CPU times: user 1.47 ms, sys: 28 μs, total: 1.49 ms
Wall time: 1.45 ms


,n.displayName,n.schemaClass,e,m.displayName,m.schemaClass,m.category
0,"E2F1:(TFDP1,TFDP2) [nucleoplasm]",Complex,20,E2F1 binds APAF1 gene promoter,Reaction,binding
1,"E2F1:(TFDP1,TFDP2) [nucleoplasm]",Complex,39,E2F1 [nucleoplasm],EntityWithAccessionedSequence,<NA>
2,APAF1:CYCS [cytosol],Complex,42,CYCS [cytosol],EntityWithAccessionedSequence,<NA>
3,APAF1:CYCS [cytosol],Complex,54,APAF1 [cytosol],EntityWithAccessionedSequence,<NA>
4,CYCS gene:NRF1:PPARGC1B [nucleoplasm],Complex,21,NRF1:PPARGC1B binds the CYCS promoter,Reaction,binding
5,CYCS gene:NRF1:PPARGC1B [nucleoplasm],Complex,22,NRF1 [nucleoplasm],EntityWithAccessionedSequence,<NA>
6,RORA:Coactivator [nucleoplasm],Complex,23,EP300 [nucleoplasm],EntityWithAccessionedSequence,<NA>
7,RORA:Coactivator [nucleoplasm],Complex,49,PPARGC1A [nucleoplasm],EntityWithAccessionedSequence,<NA>
8,"p-S15,S20-TP53:EP300:PRMT1:CARM1:GADD45A Gene ...",Complex,44,EP300 [nucleoplasm],EntityWithAccessionedSequence,<NA>
9,"p-S15,S20-TP53:EP300:PRMT1:CARM1:GADD45A Gene ...",Complex,24,"p-S15,S20-TP53 Tetramer [nucleoplasm]",Complex,<NA>


In [29]:
%%time

# Find all edges with a node of category "binding" going to an other node
command = """
MATCH (n)-[e]->(m)
WHERE n.category = "binding"
RETURN n.displayName, n.schemaClass, n.category, e, m.displayName, m.schemaClass, m.category
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    display(df)

,n.displayName,n.schemaClass,n.category,e,m.displayName,m.schemaClass,m.category
0,TP53 binds the PMAIP1 (NOXA) promoter,Reaction,binding,0,"p-S15,S20-TP53 Tetramer [nucleoplasm]",Complex,<NA>
1,TP53 binds the PMAIP1 (NOXA) promoter,Reaction,binding,1,PMAIP1 Gene [nucleoplasm],EntityWithAccessionedSequence,<NA>
2,TP53 binds the PMAIP1 (NOXA) promoter,Reaction,binding,2,"p-S15,S20-TP53 Tetramer:PMAIP1 Gene [nucleoplasm]",Complex,<NA>
3,TP53 binds the PMAIP1 (NOXA) promoter,Reaction,binding,3,TP53 stimulates PMAIP1 (NOXA) expression,BlackBoxEvent,omitted
4,TP53 binds the APAF1 gene promoter,Reaction,binding,4,TP53 stimulates APAF1 gene expression,BlackBoxEvent,omitted
5,NRF1:PPARGC1B binds the CYCS promoter,Reaction,binding,5,Expression of CYCS,BlackBoxEvent,omitted
6,NRF1:PPARGC1B binds the CYCS promoter,Reaction,binding,6,Expression of NRF1,BlackBoxEvent,omitted
7,CYCS binds to APAF1,Reaction,binding,7,Release of Cytochrome c from mitochondria,Reaction,transition
8,CYCS binds to APAF1,Reaction,binding,8,CYCS [cytosol],EntityWithAccessionedSequence,<NA>
9,E2F1 binds APAF1 gene promoter,Reaction,binding,9,APAF1 gene expression is stimulated by E2F1 an...,BlackBoxEvent,omitted


CPU times: user 4.34 ms, sys: 0 ns, total: 4.34 ms
Wall time: 4.26 ms


## Complex queries

In [30]:
def build_query_chain(
    hop_count: int,
    start_node_label: str = None,
    start_node_property: str = None,
    start_node_value: str = None,
    end_node_label: str = None,
    end_node_property: str = None,
    end_node_value: str = None,
    edge_type: str = None,
    intermediate_node_label: str = None,
    return_properties: list = None,
) -> tuple[str, list[str]]:
    """
    Build a general query to find chains between nodes.

    Parameters:
    -----------
    hop_count : int
        Number of hops/edges in the path (REQUIRED)
    start_node_label : str, optional
        Label of the starting node (e.g., 'Process', 'Station')
    start_node_property : str, optional
        Property name to match on start node (e.g., 'displayName', 'id')
    start_node_value : str, optional
        Value to match for start node
    end_node_label : str, optional
        Label of the ending node (None for any node)
    end_node_property : str, optional
        Property name to match on end node
    end_node_value : str, optional
        Value to match for end node
    edge_type : str, optional
        Type of edges to traverse (None for any edge type)
    intermediate_node_label : str, optional
        Label constraint for intermediate nodes (None for any label)
    return_properties : list, optional
        List of property names to return for nodes (default: ['displayName'])

    Returns:
    --------
    tuple : (query_string, column_names)
    """

    if return_properties is None:
        return_properties = ["displayName"]

    # Build MATCH clause
    query = "MATCH "

    # Start node - build based on what's provided
    start_parts = []
    if start_node_label:
        start_parts.append(f":{start_node_label}")
    if start_node_property and start_node_value:
        start_parts.append(f'{{{start_node_property}:"{start_node_value}"}}')

    if start_parts:
        query += f"(first{''.join(start_parts)})"
    else:
        query += "(first)"

    # Intermediate nodes and edges
    for k in range(1, hop_count + 1):
        # Edge
        if edge_type:
            query += f"-[e{k}:{edge_type}]->"
        else:
            query += f"-[e{k}]->"

        # Last hop - end node
        if k == hop_count:
            end_parts = []
            if end_node_label:
                end_parts.append(f":{end_node_label}")
            if end_node_property and end_node_value:
                end_parts.append(f'{{{end_node_property}:"{end_node_value}"}}')

            if end_parts:
                query += f"(last{''.join(end_parts)})"
            else:
                query += "(last)"
        # Intermediate nodes
        else:
            if intermediate_node_label:
                query += f"(n{k}:{intermediate_node_label})"
            else:
                query += f"(n{k})"

    # Build RETURN clause
    query += " RETURN first"
    column_names = ["first"]

    # Add start node properties
    for prop in return_properties:
        query += f", first.{prop}"
        column_names.append(f"first.{prop}")

    # Add intermediate nodes/edges
    for k in range(1, hop_count + 1):
        query += f", e{k}"
        column_names.append(f"e{k}")

        if k == hop_count:
            query += ", last"
            column_names.append("last")
            for prop in return_properties:
                query += f", last.{prop}"
                column_names.append(f"last.{prop}")
        else:
            query += f", n{k}"
            column_names.append(f"n{k}")
            for prop in return_properties:
                query += f", n{k}.{prop}"
                column_names.append(f"n{k}.{prop}")

    return query, column_names

In [31]:
%%time

# Find all paths (with maximum length of 15 hops) between a node of type "EntityWithAccessionedSequence" and a node of type "BlackBoxEvent"
max_hops = 15
longest_df = None

for hop in range(1, max_hops):
    print(100 * "*")
    print(f"{hop} hop(s) :\n")

    query, cols = build_query_chain(
        hop_count=hop,
        start_node_label="EntityWithAccessionedSequence",
        end_node_label="BlackBoxEvent",
        return_properties=["id", "displayName", "schemaClass", "category"],
    )
    #print(query)
    
    # Use with client
    df = client.query(query)
    if df.empty:
        print("No result found")
    else:
        df.columns = cols
        display(df)
        longest_df = df

print(100 * "*")

****************************************************************************************************
1 hop(s) :



,first,first.id,first.displayName,first.schemaClass,first.category,e1,last,last.id,last.displayName,last.schemaClass,last.category
0,29,CYCS [cytosol],CYCS [cytosol],EntityWithAccessionedSequence,<NA>,43,23,Expression of CYCS,Expression of CYCS,BlackBoxEvent,omitted
1,31,PMAIP1 Gene [nucleoplasm],PMAIP1 Gene [nucleoplasm],EntityWithAccessionedSequence,<NA>,46,26,TP53 stimulates PMAIP1 (NOXA) expression,TP53 stimulates PMAIP1 (NOXA) expression,BlackBoxEvent,omitted
2,31,PMAIP1 Gene [nucleoplasm],PMAIP1 Gene [nucleoplasm],EntityWithAccessionedSequence,<NA>,47,22,Transactivation of PMAIP1 (NOXA) by E2F1,Transactivation of PMAIP1 (NOXA) by E2F1,BlackBoxEvent,omitted
3,33,NRF1 [nucleoplasm],NRF1 [nucleoplasm],EntityWithAccessionedSequence,<NA>,53,24,Expression of NRF1,Expression of NRF1,BlackBoxEvent,omitted


****************************************************************************************************
2 hop(s) :



,first,first.id,first.displayName,first.schemaClass,first.category,e1,n1,n1.id,n1.displayName,n1.schemaClass,n1.category,e2,last,last.id,last.displayName,last.schemaClass,last.category
0,33,NRF1 [nucleoplasm],NRF1 [nucleoplasm],EntityWithAccessionedSequence,<NA>,51,6,"NRF1:p-PPARGC1A, NRF2 bind the TFB2M promoter","NRF1:p-PPARGC1A, NRF2 bind the TFB2M promoter",Reaction,binding,13,24,Expression of NRF1,Expression of NRF1,BlackBoxEvent,omitted
1,33,NRF1 [nucleoplasm],NRF1 [nucleoplasm],EntityWithAccessionedSequence,<NA>,52,2,NRF1:PPARGC1B binds the CYCS promoter,NRF1:PPARGC1B binds the CYCS promoter,Reaction,binding,5,23,Expression of CYCS,Expression of CYCS,BlackBoxEvent,omitted
2,33,NRF1 [nucleoplasm],NRF1 [nucleoplasm],EntityWithAccessionedSequence,<NA>,52,2,NRF1:PPARGC1B binds the CYCS promoter,NRF1:PPARGC1B binds the CYCS promoter,Reaction,binding,6,24,Expression of NRF1,Expression of NRF1,BlackBoxEvent,omitted


****************************************************************************************************
3 hop(s) :



...

****************************************************************************************************
4 hop(s) :

No result found
****************************************************************************************************
5 hop(s) :

No result found
****************************************************************************************************
6 hop(s) :

No result found
****************************************************************************************************
7 hop(s) :



,first,first.id,first.displayName,first.schemaClass,first.category,e1,n1,n1.id,n1.displayName,n1.schemaClass,...,n6.id,n6.displayName,n6.schemaClass,n6.category,e7,last,last.id,last.displayName,last.schemaClass,last.category
0,31,PMAIP1 Gene [nucleoplasm],PMAIP1 Gene [nucleoplasm],EntityWithAccessionedSequence,<NA>,47,22,Transactivation of PMAIP1 (NOXA) by E2F1,Transactivation of PMAIP1 (NOXA) by E2F1,BlackBoxEvent,...,E2F1 binds APAF1 gene promoter,E2F1 binds APAF1 gene promoter,Reaction,binding,9,21,APAF1 gene expression is stimulated by E2F1 an...,APAF1 gene expression is stimulated by E2F1 an...,BlackBoxEvent,omitted


****************************************************************************************************
8 hop(s) :

No result found
****************************************************************************************************
9 hop(s) :



,first,first.id,first.displayName,first.schemaClass,first.category,e1,n1,n1.id,n1.displayName,n1.schemaClass,...,n8.id,n8.displayName,n8.schemaClass,n8.category,e9,last,last.id,last.displayName,last.schemaClass,last.category
0,31,PMAIP1 Gene [nucleoplasm],PMAIP1 Gene [nucleoplasm],EntityWithAccessionedSequence,<NA>,46,26,TP53 stimulates PMAIP1 (NOXA) expression,TP53 stimulates PMAIP1 (NOXA) expression,BlackBoxEvent,...,E2F1 binds APAF1 gene promoter,E2F1 binds APAF1 gene promoter,Reaction,binding,9,21,APAF1 gene expression is stimulated by E2F1 an...,APAF1 gene expression is stimulated by E2F1 an...,BlackBoxEvent,omitted


****************************************************************************************************
10 hop(s) :

No result found
****************************************************************************************************
11 hop(s) :

No result found
****************************************************************************************************
12 hop(s) :

No result found
****************************************************************************************************
13 hop(s) :

No result found
****************************************************************************************************
14 hop(s) :

No result found
****************************************************************************************************
CPU times: user 62.3 ms, sys: 2 ms, total: 64.3 ms
Wall time: 64.6 ms


# Create subgraph to visualise

In [32]:
# Get subgraph
subset_nodes = longest_df.filter(regex="id$", axis=1).iloc[0].values.tolist()
subG = G.subgraph(subset_nodes).copy()
print(subG)

# Build CREATE command from subgraph

MultiGraph with 10 nodes and 10 edges


In [33]:
subgraph_name = f"{graph_name}_subgraph"
subgraph_name

'reactome3_subgraph'

In [34]:
%%time

start_time = time.time()

# Write subgraph to JSONL format
print("Writing subgraph to JSONL...")
jsonl_filename = networkx_to_jsonl(subG, subgraph_name)

# Load JSONL into TuringDB
print(f"\nLoading JSONL into TuringDB as graph '{subgraph_name}' ...")
client.query(f"LOAD JSONL '{jsonl_filename}' AS {subgraph_name}")

execution_time = time.time() - start_time
print(f"\n\u2713 Graph '{subgraph_name}' created in {execution_time:.2f} seconds")

# Set active graph for querying
client.set_graph(subgraph_name)

Writing subgraph to JSONL...
JSONL file written to: /home/ubuntu/.turing/data/reactome3_subgraph.jsonl
Graph: 10 nodes, 10 edges

Loading JSONL into TuringDB as graph 'reactome3_subgraph' ...

✓ Graph 'reactome3_subgraph' created in 0.07 seconds
CPU times: user 267 μs, sys: 1.98 ms, total: 2.24 ms
Wall time: 68.9 ms


<div class="alert alert-block alert-info">
    <h2>
        You can visualise the subgraph directly in the notebook below.
    </h2>
</div>

In [35]:
from pyvis.network import Network
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

net = Network(
    height="750px",
    width="100%",
    notebook=True,
    bgcolor="#ffffff",
    font_color="#000000",
    directed=True,
)

# Choose your palette (can be changed easily)
palette_name = (
    "Pastel1"  # Options: 'tab10', 'Set3', 'Paired', 'viridis', 'plasma', etc.
)

# Get unique node types
unique_types = list(set(data.get("schemaClass") for _, data in G.nodes(data=True)))

# Get colors from matplotlib palette
cmap = plt.get_cmap(palette_name)
colors = [
    mcolors.rgb2hex(cmap(i / len(unique_types))) for i in range(len(unique_types))
]

# Map types to colors
type_colors = {node_type: colors[i] for i, node_type in enumerate(unique_types)}

# Then use in your visualization
for node, data in subG.nodes(data=True):
    node_type = data.get("schemaClass", "Unknown")
    color = type_colors.get(node_type, "#95a5a6")

    net.add_node(
        node,
        label=data.get("displayName", str(node)),
        title=f"{data.get('displayName', '')}",
        color=color,
        size=25,
    )

for source, target, data in subG.edges(data=True):
    net.add_edge(source, target, color="#95a5a6", width=3)

net.toggle_physics(status=True)
net.show(f"{example_name}_subgraph.html")

reactome_subgraph.html


# Use LLM to generate Cypher query

Before running this section, create a `.env` file in the project root with your API keys:

```env
ANTHROPIC_API_KEY=your_key_here
OPENAI_API_KEY=your_key_here
MISTRAL_API_KEY=your_key_here

In [36]:
client.set_graph(graph_name)

In [37]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
if not load_dotenv():
    raise ValueError("LLM credentials not saved in .env file")

In [38]:
api_keys = {
    "Anthropic": os.getenv("ANTHROPIC_API_KEY"),
    "Mistral": os.getenv("MISTRAL_API_KEY"),
    "OpenAI": os.getenv("OPENAI_API_KEY"),
}

In [39]:
"""Build system prompt with TuringDB schema and examples"""

turingdb_cypher_system_prompt = """
You are an expert at converting natural language questions into TuringDB queries.

Your task is to generate syntactically correct TuringDB queries based on natural language input.

VERY IMPORTANT - YOU MUST FOLLOW THESE REQUESTS - TuringDB Syntax Guidelines:
1. Return ONLY the TuringDB query, no explanations or markdown formatting
2. Use MATCH, CREATE and WHERE operations only
3. Nodes: (n:Label {property = "value"}) or (n:Label {property: value})
4. Edges: Use DIRECTED syntax with ->
5. Pattern matching: MATCH (n)-[e]->(m)
6. Property matching: Use = operator for exact matching
7. Multiple constraints: (n:Person:Engineer {name = "John", age = 30})
8. Return all matched entities: RETURN n, e, m or use RETURN * for all
9. Filter using WHERE clause: MATCH (n:Person) WHERE n.name = 'John' RETURN n.firstname, n.lastname

VERY IMPORTANT - YOU ARE NOT ALLOWED TO USE THE FOLLOWING - FORBIDDEN in TuringDB:
- Do NOT use AS aliases
- Do NOT use LIMIT, SKIP clauses
- Do NOT use WITH clauses
- Do NOT use CALL (except for metaqueries)
- Do NOT use toLower() or other functions
- Do NOT use wildcard character (*)
- Do NOT use multi-hops pattern for edges: e.g. `-[e:CONNECTED*1..10]->`
- Do NOT use "end" or "s3" variable name

Supported TuringDB Operations:
- MATCH queries: MATCH (n:Label)-[e:Type]->(m) RETURN n, m
- CREATE queries: CREATE (n:Label{property="value"})-[e:Type]->(m:Label)
- Metaqueries: CALL db.propertyTypes(), CALL db.labels(), CALL db.edgeTypes()
- Property types: String ("text" or `text`), Boolean (true/false), Integer (20), Double (20.5)

Examples for few-shot learning:
- Find all persons: MATCH (n:Person) RETURN n
- Find connections: MATCH (n:Person)-[e]->(m:Person) RETURN n, e, m
- Create person: CREATE (n:Person{name="John", age=30})
- Match person with specific name: MATCH (p:Person) WHERE p.name = "John" RETURN p
- Path with 1 hop between Station Paddington and Blackfriars:  MATCH (first:Station{displayName:"Paddington"})-[e1:CONNECTED]->(last:Station{displayName="Blackfriars"}) RETURN start, start.displayName, start.Note, e1.Line, last, last.displayName, last.Note
- Path with 2 hops between Station Paddington and Blackfriars: MATCH (first:Station{displayName:"Paddington"})-[e1:CONNECTED]->(s1:Station)-[e2:CONNECTED]->(last:Station{displayName="Blackfriars"}) RETURN start, start.displayName, start.Note, e1.Line, s1, s1.displayName, s1.Note, e2.Line, last, last.displayName, last.Note
- Path with 8 hops between Station Paddington and Blackfriars: MATCH (first:Station{displayName:"Paddington"})-[e1:CONNECTED]->(s1:Station)-[e2:CONNECTED]->(s2:Station)-[e3:CONNECTED]->(s3:Station)-[e4:CONNECTED]->(s4:Station)-[e5:CONNECTED]->(s5:Station)-[e6:CONNECTED]->(s6:Station)-[e7:CONNECTED]->(s7:Station)-[e8:CONNECTED]->(last:Station{displayName="Blackfriars"}) RETURN start, start.displayName, start.Note, e1.Line, s1, s1.displayName, s1.Note, e2.Line, s2, s2.displayName, s2.Note, e3.Line, s3, s3.displayName, s3.Note, e4.Line, s4, s4.displayName, s4.Note, e5.Line, s5, s5.displayName, s5.Note, e6.Line, s6, s6.displayName, s6.Note, e7.Line, s7, s7.displayName, s7.Note, e8.Line, last, last.displayName, last.Note
- Find all Chinese providers and what they supply: MATCH (n{provider_country:"CHN"}) RETURN n, n.provider_name, n.displayName, n.share_provided, n.type
- Find all deposition tools and their types: MATCH (specific)-[e:IS_TYPE_OF]->(general:Tool_Resource{displayName:"Deposition tools"}) RETURN specific, specific.displayName, specific.provider_name, e, general, general.displayName
"""

In [40]:
# Create system_prompt
system_prompt = f"""
TuringDB Cypher prompt :
{turingdb_cypher_system_prompt}

Here is a sample of nodes from the graph so you know the structure and available properties :
{client.query("MATCH (n) RETURN n LIMIT 3")}

Here is the output of "CALL LABELS ()" command, showing the different node types of the graph :
{client.query("CALL db.labels()")}

Here is also the output of "CALL EDGETYPES ()" command, showing the different edge types of the graph :
{client.query("CALL db.edgeTypes()")}

Very important :
- You MUST follow current TuringDB Syntax Guidelines
- You MUST NOT USE what is FORBIDDEN in TuringDB
- By default, RETURN ALL THE MATCHED NODES AND EDGES AND THEIR PROPERTIES in the RETURN section (except contrary demand from user)
- Use the correct node and edge properties name in the MATCH section.
- Use the correct node and edge properties name in the RETURN section.
- Pay attention to which properties come from nodes or edges, to create a functioning query
- Pay attention to lower and uppercases in properties
- If some properties contain spaces, be careful to wrap them

Give me the query FOLLOWING TURINGDB GUIDELINES AND NOT USING WHAT IS FORBIDDEN for this specific question :
"""


In [40]:
"""Build system prompt with TuringDB schema and examples"""

turingdb_cypher_system_prompt = """
You are an expert at converting natural language questions into TuringDB queries.

Your task is to generate syntactically correct TuringDB queries based on natural language input.

VERY IMPORTANT - YOU MUST FOLLOW THESE REQUESTS - TuringDB Syntax Guidelines:
1. Return ONLY the TuringDB query, no explanations or markdown formatting
2. Use MATCH, CREATE and WHERE operations only
3. Nodes: (n:Label {property: value})
4. Edges: Use DIRECTED syntax with ->
5. Pattern matching: MATCH (n)-[e]->(m)
6. Property matching: Use : operator for exact matching
7. Multiple constraints: (n:Person:Engineer {name: "John", age: 30})
8. Return all matched entities: RETURN n, e, m or use RETURN * for all
9. Filter using WHERE clause: MATCH (n:Person) WHERE n.name = 'John' RETURN n.firstname, n.lastname

VERY IMPORTANT - YOU ARE NOT ALLOWED TO USE THE FOLLOWING - FORBIDDEN in TuringDB:
- Do NOT use AS aliases
- Do NOT use LIMIT, SKIP clauses
- Do NOT use WITH clauses
- Do NOT use CALL (except for metaqueries)
- Do NOT use toLower() or other functions
- Do NOT use wildcard character (*)
- Do NOT use multi-hops pattern for edges: e.g. `-[e:CONNECTED*1..10]->`
- Do NOT use "end" or "s3" variable name

Supported TuringDB Operations:
- MATCH queries: MATCH (n:Label)-[e:Type]->(m) RETURN n, m
- CREATE queries: CREATE (n:Label{property:"value"})-[e:Type]->(m:Label)
- Metaqueries: CALL db.propertyTypes(), CALL db.labels(), CALL db.edgeTypes()
- Property types: String ("text" or `text`), Boolean (true/false), Integer (20), Double (20.5)

Examples for few-shot learning:
- Find all persons: MATCH (n:Person) RETURN n
- Find connections: MATCH (n:Person)-[e]->(m:Person) RETURN n, e, m
- Create person: CREATE (n:Person {name: "John", age: 30})
- Match person with specific name: MATCH (p:Person) WHERE p.name = "John" RETURN p
- Path with 1 hop between Station Paddington and Blackfriars:  MATCH (departure:Station{displayName:"Paddington"})-[c1:CONNECTED]->(arrival:Station{displayName:"Blackfriars"}) RETURN start, start.displayName, start.Note, ec.Line, last, last.displayName, last.Note
- Path with 2 hops between Station Paddington and Blackfriars: MATCH (departure:Station{displayName:"Paddington"})-[c1:CONNECTED]->(st1:Station)-[c2:CONNECTED]->(arrival:Station{displayName:"Blackfriars"}) RETURN start, start.displayName, start.Note, c1.Line, s1, s1.displayName, s1.Note, c2.Line, last, last.displayName, last.Note
- Path with 8 hops between Station Paddington and Blackfriars: MATCH (departure:Station{displayName:"Paddington"})-[c1:CONNECTED]->(st1:Station)-[c2:CONNECTED]->(st2:Station)-[c3:CONNECTED]->(st3:Station)-[c4:CONNECTED]->(st4:Station)-[c5:CONNECTED]->(st5:Station)-[c6:CONNECTED]->(st6:Station)-[c7:CONNECTED]->(st7:Station)-[c8:CONNECTED]->(arrival:Station{displayName:"Blackfriars"}) RETURN start, start.displayName, start.Note, c1.Line, st1, st1.displayName, st1.Note, c2.Line, st2, st2.displayName, st2.Note, c3.Line, st3, st3.displayName, st3.Note, c4.Line, st4, st4.displayName, st4.Note, c5.Line, st5, st5.displayName, st5.Note, c6.Line, st6, st6.displayName, st6.Note, c7.Line, st7, st7.displayName, st7.Note, c8.Line, last, last.displayName, last.Note
- Find all Chinese providers and what they supply: MATCH (n{provider_country:"CHN"}) RETURN n, n.provider_name, n.displayName, n.share_provided, n.type
- Find all deposition tools and their types: MATCH (specific)-[e:IS_TYPE_OF]->(general:Tool_Resource{displayName:"Deposition tools"}) RETURN specific, specific.displayName, specific.provider_name, e, general, general.displayName

## SCHEMA CONTEXT
{schema_context}
"""

In [41]:
# Build schema context for LLM
labels_result = client.query("CALL db.labels()")
rel_types_result = client.query("CALL db.edgeTypes()")
prop_types_result = client.query("CALL db.propertyTypes()")
query_sample_nodes = "MATCH (n) RETURN " + ", ".join([f"n.`{prop}`" for prop in prop_types_result["propertyType"].tolist()]) + "LIMIT 3"
sample_nodes = client.query(query_sample_nodes)

schema_context = f"""Node labels: {labels_result}
Relationship types: {rel_types_result}
Property names and types: {prop_types_result}
Sample nodes: {sample_nodes}"""

system_prompt = turingdb_cypher_system_prompt.replace("{schema_context}", schema_context)

In [42]:
question = """
Is there a path of any size linking gene PMAIP1 and gene E2F1 ?
"""

In [43]:
%%time

provider = "OpenAI"

cypher_query = natural_language_to_cypher(
    question=question,
    system_prompt=system_prompt,
    provider=provider,
    api_key=api_keys[provider],
    temperature=0.0,
)
print(f"cypher_query : {cypher_query}")

cypher_query : MATCH (gene1:EntityWithAccessionedSequence {name: "PMAIP1"})-[e:CONNECTED]-(gene2:EntityWithAccessionedSequence {name: "E2F1"}) RETURN gene1, e, gene2
CPU times: user 385 ms, sys: 52.7 ms, total: 438 ms
Wall time: 4.16 s


In [44]:
df_path = client.query(cypher_query)
if df_path.empty:
    print("--> No result found\n")
else:
    display(df_path)

--> No result found



# Use LLM to get subgraph summary

In [45]:
%%time

prompt = f"""
Give me a summary of this graph. It represents biological entities, tell me more about the entities involved and the interactions.
Here is the graph :
{G.nodes(data=True)} {G.edges(data=True)}
"""

system_prompt = """
You are a specialist in analysing graphs and their structure.
You will use your knowledge to add more information about the entities and relationships in the graph.
Add information only when you are sure it is relevant.
"""

provider = "OpenAI"

response = query_llm(
    prompt=prompt,
    system_prompt=system_prompt,
    provider=provider,
    api_key=api_keys[provider],
    temperature=0.0,
)

CPU times: user 7.49 ms, sys: 1.98 ms, total: 9.46 ms
Wall time: 30.6 s


In [46]:
display(Markdown(response))

The graph represents a complex network of interactions involving various biological entities primarily related to apoptosis and gene regulation in *Homo sapiens*. Here’s a summary of the key entities and their interactions:

### Key Entities:
1. **TP53 (Tumor Protein p53)**: A crucial tumor suppressor protein that regulates the cell cycle and functions in preventing cancer. It binds to the promoters of various genes, including PMAIP1 and APAF1, to stimulate their expression.
   - **p-S15,S20-TP53 Tetramer**: A phosphorylated form of TP53 that is active in the nucleoplasm and involved in gene regulation.

2. **PMAIP1 (NOXA)**: A pro-apoptotic gene that is regulated by TP53. Its expression is crucial for apoptosis, particularly in response to cellular stress.
   - **PMAIP1 Gene**: The DNA sequence that encodes the PMAIP1 protein.

3. **APAF1 (Apoptotic Protease Activating Factor 1)**: A protein that plays a key role in the apoptosome formation, which is essential for the activation of caspases during apoptosis.
   - **APAF1 Gene**: The gene encoding the APAF1 protein.

4. **CYCS (Cytochrome c)**: A protein involved in the electron transport chain and apoptosis. It is released from mitochondria during apoptosis and interacts with APAF1 to promote apoptosome formation.

5. **E2F1**: A transcription factor that regulates the expression of genes involved in cell cycle progression and apoptosis. It binds to the promoters of PMAIP1 and APAF1, stimulating their expression.

6. **NRF1 (Nuclear Respiratory Factor 1)**: A transcription factor that regulates the expression of genes involved in mitochondrial biogenesis and function, including CYCS.

7. **PPARGC1A (Peroxisome Proliferator-Activated Receptor Gamma Coactivator 1-alpha)**: A coactivator that regulates genes involved in energy metabolism and mitochondrial biogenesis.

8. **ESRRA (Estrogen-Related Receptor Alpha)**: A transcription factor that regulates various genes, including those involved in energy metabolism.

### Key Interactions:
- **TP53 Binding**: TP53 binds to the promoters of PMAIP1 and APAF1, stimulating their expression, which is critical for apoptosis.
- **Complex Formation**: The p-S15,S20-TP53 tetramer forms complexes with PMAIP1 and other regulatory proteins, enhancing gene expression.
- **CYCS and APAF1 Interaction**: Cytochrome c binds to APAF1, leading to the release of cytochrome c from mitochondria, a key step in the apoptotic pathway.
- **E2F1 Regulation**: E2F1 binds to the promoters of PMAIP1 and APAF1, promoting their transcription, and is involved in positive regulation of gene expression.
- **NRF1 and CYCS**: NRF1 binds to the CYCS promoter, regulating its expression, which is essential for mitochondrial function and apoptosis.
- **Positive Regulation**: Various complexes, including those involving ESRRA and PPARGC1A, positively regulate the expression of genes involved in apoptosis and mitochondrial function.

### Conclusion:
The graph illustrates a network of interactions that highlight the regulatory roles of TP53, E2F1, and NRF1 in apoptosis and gene expression. The interplay between these proteins and their target genes is crucial for maintaining cellular homeostasis and responding to stress signals, ultimately influencing cell fate decisions such as survival or apoptosis.

In [49]:
print("Notebook finished !")

Notebook finished !
